In [ ]:
import numpy as np
import pandas as pd
 
df = pd.read_csv('C:/Users/Orestis/Desktop/1.Diplomatiki/Data/Item Data/imagenewsreel-itemData.csv', delimiter=';', header=None, escapechar='\\', quotechar='\"')
df.columns = ['id','imageId','title','publisher','url','image_url','stemmed','date','article_id']
df

In [2]:
df.isnull().sum()
#se poies stiles loipin times kai poses stin kahte mia, 2:titlos,6:stemmed text

id             0
imageId        0
title         26
publisher      0
url            0
image_url      0
stemmed       29
date           0
article_id     0
dtype: int64

In [ ]:
#poies einai oi grammes poy exoun Null sthn 2 stili
df[df[2].isnull()]

In [7]:
#sbinw tis NaN
nonull_df = df.dropna(how='any', axis = 0)
nonull_df.shape

(51368, 9)

In [8]:
##arithmos me duplicated article id
nonull_df.duplicated(subset = 'article_id').sum()

108

In [9]:
##akoma enas tropos einai 
nonull_df.article_id.duplicated().sum()

108

In [10]:
##afairw tis 108 diploeggrafes twn article_id
final_df = nonull_df.drop_duplicates(subset = 'article_id')
final_df.shape,type(final_df)
###allazei kai to nonull_df

((51260, 9), pandas.core.frame.DataFrame)

In [11]:
#krataw toyw titlous
my_titles = final_df['title']
len(my_titles)

51260

In [12]:
my_titles[1]

'Chri\xadsto\xadpher Lauer über die innere Sicher\xadheit sowie Sinn und Zweck der Video\xadüber\xadwa\xadchung am Südkreuz.'

In [13]:
#lower case
my_titles = my_titles.str.lower()
my_titles

0        in schö­ne­berg warf ein junger mann in der ve...
1        chri­sto­pher lauer über die innere sicher­hei...
2        die wähler in kata­lo­nien haben die spani­sch...
3        nach mehr als drei­jäh­riger bauzeit hat an de...
4        elek­tro­ni­sche musik hat nun auch myanmar er...
5        die spek­ta­ku­läre rück­kehr von jupp heyn­ck...
6        yahya a. schnitt seiner frau die kehle durch -...
7        jürgen b. harder sitzt in einem hessi­schen ge...
8        immer wieder kommt es in schö­ne­berg im kiez ...
9        ein 16-jäh­riger und ein 20-jäh­riger stachen ...
10       die ermittler kamen dem täter mit einem dna-te...
11       mehr­fach wurde der tod des berliner ex-rap­pe...
12       nach wohnungs­brand sprang mann aus dem 9. sto...
13       zum ersten mal nach der kölner silves­ter­nach...
14       in berlin wurde am sonn­abend ein super­markt ...
15       der 23-jäh­rige ahmed a. , der seine ehema­lig...
16       wer am montag auf die männer geschossen hat un.

In [140]:
###thelw na ftiaksw ton pinaka words_list pou periexei oles tis lekseis twn titlwn kahte arthrou ksexwrista
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'): 
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
words_list = []
stop_words = set(nltk.corpus.stopwords.words(['german']))
for x in my_titles:
    temp = []
    tokens = tokenizer.tokenize(x.replace(u'\xad', u''))   #afairw ton \xad xaraktira
    tagged_sentence = nltk.pos_tag(tokens)
    for word, tag in tagged_sentence:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag:
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        else:
            lemma = word
        if lemma not in stop_words and len(lemma) > 2 and not lemma.isdigit():   #kanw exclude ta stopwords tis mikres lekseis-arithmous kai tou arithmous se morfi leksis
            temp.append(lemma)
    words_list.append(temp)

In [143]:
len(words_list)

51260

In [144]:
### Kane save ton pinaka words_list gia na min trexw ton lemmatizer sunexeia
import pickle

with open("lemmatized_titles.pk", 'wb') as handle:
    pickle.dump(words_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
### FORTWNW KATEUTHIAN TO WORDS_LIST GIA NA MIN KANW LEMMATIZE KATHE FORA 12 LEPTA!!!!!!
import pickle

with open('lemmatized_titles.pk', 'rb') as handle:
    words_list = pickle.load(handle)
len(words_list)  ###prepei na einai o arithmos twn arthrwn

51260

In [25]:
import io
import unidecode

embedding_voc = {}
with open('C:/Users/Orestis/Desktop/cc.de.300.vec', encoding='utf-8') as embedding_file:
    for i, l in enumerate(embedding_file.readlines()):
        values = l.rstrip().rsplit(' ')     #removing whitespaces kai split
        word = unidecode.unidecode(values[0])
        embedding_voc[values[0]] = i
        # coefs = np.asarray(values[1:], dtype='float32')
        # embedding_matrix[i] = coefs
# exw se morfi dict to pretrained vocabulary

In [26]:
len(embedding_voc)

2000000

In [15]:
### Dimiourgia vocabulary me oles tis lekseis twn titlwn...xrisimopoiw word tokenizer apo nltk
# import nltk
# from nltk import word_tokenize
# from nltk.tokenize import RegexpTokenizer

my_voc = {}
counter = 0
for words in words_list:    #gia kathe arhtro/titlo
    for s in words:         #gia kathe leksi
        if s not in my_voc:
            my_voc[s] = counter
            counter += 1
len(my_voc)

86241

In [16]:
with open("my_vocabulary.pk", 'wb') as handle:
    pickle.dump(my_voc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# ###swzw tin lista me tis tokenized lekseis ana arthro gia na min xreiazetai na to kanw sunexeia
# words_list = []my
# for x in my_titles:
#     words_list.append(tokenizer.tokenize(x.replace(u'\xad', u'')))
    

In [11]:
print(type(words_list), len(words_list))
###to megethos tha prepei na einai o arithmos twn arthrwn

<class 'list'> 51368


In [17]:
#megistos arithmos leksewn se titlo
max_words = 0
thesi = 0
for k,x in enumerate(words_list):
    counter = 0
    for words in x:
        counter += 1
        if counter > max_words:
            max_words = counter
            thesi = k
print(max_words,thesi)

29 32686


In [148]:
words_list[32686]

['aufgeregt',
 'laufen',
 'hunderte',
 'ferkel',
 'verschläge',
 'erst',
 'wenige',
 'tage',
 'alte',
 'frischlinge',
 'suchen',
 'nahrung',
 'muttersau',
 'eber',
 'boris',
 'döst',
 'derweil',
 'stall',
 'gut',
 'aparte',
 'höfe',
 'ziehen',
 'michael',
 'patrick',
 'gülden',
 'seit',
 'frühsommer',
 'ferkel',
 'heran']

In [28]:
#thelw na dw poses fores emfanizetai i kathe leksi se olous tous titlous twn arthrwn
sum_dict = {}

for x in words_list:
    for word in x:
        if word not in sum_dict:
            sum_dict[word] = 1
        else:
            sum_dict[word] = sum_dict[word] + 1

len(sum_dict)
#idio megethos me to vocabulary ara komple

86241

In [29]:
###twra thelw na kratisw tis 20000 most used lekseis.epomenws tha kanw sort sto sum_dict apo to megalutero key pros to mikrotero kai tha parw
# tis prwtes 20000 thesis
import operator
sorted_list = sorted(sum_dict.items(), key=operator.itemgetter(1), reverse = True)
#to sorted_dict einai mia lista me tuples kai oxi dictionary

most_used = []
for x in range(0,20000):
    most_used.append(sorted_list[x][0])
most_used = set(most_used)
### epomenws exw mia lista most_used me tis perissotero xrisimopoihmenes lekseis
len(most_used)

20000

In [ ]:
#print(most_used)

In [27]:
## ksanakanw ena vocabulary gia ton pinaka twn most_used leksewn kai oxi olwn
import unidecode

counter = 1  
mostused_voc={}
for x in words_list:
    for word in x:
        word = unidecode.unidecode(word)
        if (word not in mostused_voc) and (word in embedding_voc):   #extra: theloume vocabulary twn most_used leksewn pou uparxoun kai sto
            mostused_voc[word] = counter
            counter+=1
len(mostused_voc)   ###to swzw ws vocabulary_emb.pk

38060

In [27]:
# lista me tous kwdikous tou vocabulary twn pio xrisimopoihmenwn leksewn tou kathe article
word_codes = []
for x in words_list:
    numbers = []
    for word in x:
        if word in mostused_voc:
            numbers.append(mostused_voc[word])  ## me auto ton tropo exoume touw kwsikous twn most_used leksewn kathe arthrou alla oxi thn 
                                                ## seira emfanisis tous, alla dn mas indiaferei
        
    for i in range(29 - len(numbers)):
        numbers.append(0)       ##o kwdikos tou extra(<PAD>) xaraktira, wste na exoun oloi oi titloi to idio mikos
            
    word_codes.append(numbers)

len(word_codes)   

51260

In [28]:
##apothikeuw ton pinaka word_codes
with open("word_codes_emb.pk", 'wb') as handle:
    pickle.dump(word_codes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
with open("vocabulary_emb.pk", 'wb') as handle:
    pickle.dump(mostused_voc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
### twra kanw ena dictionary me value to article_id, kai key ta word_codes...kai meta tha prosthesw san keys kai ta annotations kai tou
##publishers

In [6]:
article_id = final_df['article_id']
len(article_id)

51260

In [8]:
import pickle
with open("article_id.pk", 'wb') as handle:
    pickle.dump(article_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
articles_info = {}
for ids, codes in zip(article_id,word_codes):
    articles_info[ids] = codes

len(articles_info)
# dict(list(articles_info.items())[0:2])      
# articles_info[435053444] 
    

In [156]:
type(article_id),type(word_codes)

(pandas.core.series.Series, list)

In [172]:
with open("articles_info.pk", 'wb') as handle:
    pickle.dump(articles_info, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [1]:
##dimiourgia pinana embeddings gia tis most used lekseis
H_embeddings = np.random.rand(len(my_voc)+1,300)
H_embeddings

NameError: name 'np' is not defined

In [59]:
#kanw load to vocabulary tou Glove
import csv

glov_voc = {}
inputfile = pd.read_csv('C:/Users/Orestis/Desktop/1.Diplomatiki/Glove_Voc/vocab.txt', header=None, sep=' ')
c = 0
for row in inputfile[0].tolist():
    glov_voc[row] = c
    c += 1
# inf = open('C:/Users/Orestis/Desktop/1.Diplomatiki/Glove_Voc/vocab.txt', 'r')
# print(inf.read())

In [60]:
a = glov_voc.keys()
list(a)[:10]

['der', 'und', 'die', 'in', 'von', 'im', 'des', 'den', 'kategorie', 'mit']

In [91]:
len(words_list)

51368

In [93]:
import pickle

with open("lemmatized_titles.pk", 'wb') as handle:
    pickle.dump(words_list, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [62]:
import nltk

test = 'This is a, test tokenization\ problem.'
tokens = word_tokenize(test)
tokens

list

In [106]:
nonull_df.duplicated(subset = [nonull_df[8], nonull_df[2]])   ### den douleuei..prepei na prosthesw headers sta columns

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [20]:
a = 'eight'
b = a.isdigit()
b

False